In [4]:
import os
import shutil
from typing import Any
import requests
import re
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from nltk.tokenize import word_tokenize
import nltk
from collections import Counter
import torch
import torch.nn as nn
from torch.nn import functional as F
import pickle
from tqdm import tqdm
from d2l import torch as d2l
nltk.download('punkt')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\nguye\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [5]:
class Vocab:
    def __init__(self, tokens = [], min_freq = 0, reserved_tokens = []):
        self._build(tokens, min_freq, reserved_tokens)
    
    def _build(self, tokens, min_freq, reserved_tokens):
        print(f'building vocab from {len(tokens)} tokens')
        counter = Counter(tokens)
        self.token_freq = sorted(counter.items(), key = lambda x: x[1], reverse = True)

        self.idx_to_tokens = list(sorted(set(['<unk>'] + reserved_tokens \
        + [ token for token, freq in self.token_freq if freq > min_freq])))
    
        self.token_to_idx = {token: idx for idx, token in enumerate(self.idx_to_tokens)}

        print('built vocab object')

    def __len__(self):
        return len(self.idx_to_tokens)
    
    def __getitem__(self, tokens):
        #if not type list or tuple
        if not isinstance(tokens, (list, tuple)):
            return self.token_to_idx.get(tokens, self.unk)
        return [self.__getitem__(t) for t in tokens]
    
    def to_tokens(self, idx):
        if not isinstance(idx, (list, tuple)):
            return self.idx_to_tokens[idx]
        return [self.to_tokens(i) for i in idx]
    
    @property
    def unk(self):
        return self.token_to_idx['<unk>']

class ProjectGutenbergDataset(Dataset):
    def _download(self, links_path: str = 'links.txt', output_dir: str = 'dataset') -> None:
        '''
        iterate through links in links.txt in Project Gutenberg to download books
        '''


        #read links from file
        if os.path.exists(output_dir) == False:
            print('Downloading books ... ')
            books = []
            try:
                with open(links_path, 'r') as file:
                    errors = []
                    for link in file.readlines():
                        link = link.rstrip()

                        res = requests.get(link)
                        if res.status_code != 200:
                            raise Exception(f"Failed to Fetch, Error code {res.status_code}")
                        books.append(res.text)
                        print(f"SUCCESS {link}")

                if os.path.exists(output_dir) == False:
                    os.mkdir(output_dir) 

                for id, book in enumerate(books):
                    output_path = os.path.join(output_dir, f'book{id}.txt')
                    with open(output_path, 'w') as file:
                        file.write(book)

            except Exception as e:
                print('Error while downloading books, error = ', e)
        else:
            print("PG dataset loaded")


    def _preprocess(self, text):
        #remove digits and anything but letters and space
        return re.sub('[^a-zA-Z\s]', '', text).lower()
    
    def _tokenize(self, tokenizer, text: str, save_to_file = False) -> list[str]:
        tokens = tokenizer(self._preprocess(text))
        # save tokens
        if save_to_file:
            with open('tokens.txt', 'w') as file:
                for t in tokens:
                    file.writelines(f'{t} \n')

        return tokens

    def _build(self, dataset_path):
        '''
        @param:
            dataset_path: str, path to PG dataset
        @return
            corpus: list[int] 
            vocab: Vocab object
        '''
        alltext = ''
        for file in os.listdir(dataset_path):
            filepath = os.path.join(dataset_path, file)
            with open(filepath, 'r') as file:
                alltext += file.read()
    
        #init tokenizer
        tokenizer = word_tokenize

        tokens = self._tokenize(tokenizer, alltext, save_to_file=True)
        
        vocab = Vocab(tokens, min_freq = 2)
        
        #build corpus, list of indices, [1, 2,100,44,33,...] 
        corpus = [vocab[token] for token in tokens]

        return corpus, vocab

    def __init__(self, dataset_path = './dataset/', num_steps = 100):
        self.num_steps = num_steps 

        self._download()
        corpus, vocab = self._build(dataset_path)

        #save the corpus and vocab for inspection later
        with open('corpus.pkl', 'wb') as file:
            pickle.dump(corpus, file)

        with open('corpus.txt', 'w') as file:
            for token in corpus: 
                file.write(str(token) + " ")
        
        with open('vocab_obj.pkl', 'wb') as file:
            pickle.dump(vocab, file)

        with open('vocab_obj.txt', 'w') as file:
            for token, idx in vocab.token_to_idx.items(): 
                file.writelines(f'{token}:{idx}\n')

        N = len(corpus)

        array = torch.tensor([corpus[i : i + num_steps + 1] for i in range(N - num_steps)])
        self.X = array[:,:-1] 
        self.Y = array[:,1:] 
        self.vocab = vocab
 
    def __len__(self) -> int:
        return len(self.X)

    def __getitem__(self, index: int) -> Any:
        return self.X[index], self.Y[index]

# dataset = ProjectGutenbergDataset()

# feature, label = dataset[0]

In [6]:
# print(len(dataset))
# print(feature)
# print(len(feature))
# print(label)
# print(len(label))

In [7]:
# with open('vocab_obj.pkl', 'rb') as file:
#     vocab = pickle.load(file)

# print(vocab.to_tokens(feature.tolist()))
# print(vocab.to_tokens(label.tolist()))

### Try loading the corpus pkl

In [8]:
# with open('corpus.pkl', 'rb')  as file:
#     corpus = pickle.load(file)

# print(type(corpus))
# print(corpus[:10])
# print(vocab.to_tokens(corpus[500:600]))

# Dataloader

# RNN and LM from d2l to establish baselines

In [9]:
a = torch.tensor([[1,2,3,4], [2,3,4,5]])
print(a.shape)
b = torch.tensor([[9,8,7,6], [7,6,5,4]])
print(b.shape)
c = torch.stack((a,b), 0)
print(c.shape)
c = torch.stack((a,b), 1)
print(c.shape)
c = torch.stack((a,b), 2)
print(c)
print(c.shape)

torch.Size([2, 4])
torch.Size([2, 4])
torch.Size([2, 2, 4])
torch.Size([2, 2, 4])
tensor([[[1, 9],
         [2, 8],
         [3, 7],
         [4, 6]],

        [[2, 7],
         [3, 6],
         [4, 5],
         [5, 4]]])
torch.Size([2, 4, 2])


In [10]:
def get_device():
    return 'cuda' if torch.cuda.device_count() else 'cpu'

In [11]:
class RNN(nn.Module):
    def __init__(self, num_inputs, num_hiddens,sigma = 0.001):
        super().__init__()
        self.num_inputs = num_inputs
        self.num_hiddens = num_hiddens
        self.device = get_device()
        self.Wxh = nn.Parameter(torch.rand((num_inputs, num_hiddens), dtype = torch.float, device = self.device) * sigma)
        self.Whh = nn.Parameter(torch.rand((num_hiddens, num_hiddens), dtype = torch.float, device = self.device) * sigma)
        self.bh = nn.Parameter(torch.rand((1, num_hiddens), dtype = torch.float, device = self.device) * sigma)
    
    def forward(self, inputs, state = None):
        # N is num steps
        # n is batch size
        # d is num inputs
        N, n, d = inputs.shape
        if state == None:
            #state is not Parameter and will not be used for backprop, initalize state with 0s
            state = torch.zeros((n, self.num_hiddens), device = self.device)
        else:
            state, = state

        outputs = []

        for X in inputs:
            
            state = torch.tanh(X @ self.Wxh + state @ self.Whh + self.bh)
            outputs.append(state)
        
        outputs = torch.stack(outputs, 0)
        return outputs, state

#params
num_steps = 100
batch_size = 8
num_inputs = 16
num_hiddens = 32

sample_inputs = torch.rand((num_steps, batch_size, num_inputs), device = get_device())
print('sample input shape = ',sample_inputs.shape)
#Testing RNN correctness
rnn = RNN(num_inputs, num_hiddens)
outputs, state = rnn(sample_inputs)
print(outputs.shape)
print(state.shape)

    

sample input shape =  torch.Size([100, 8, 16])
torch.Size([100, 8, 32])
torch.Size([8, 32])


In [12]:
class LanguageModel(nn.Module):

    def __init__(self, rnn, vocab_size, sigma = 0.001):
        super().__init__()
        self.rnn = rnn
        self.vocab_size = vocab_size
        self.sigma = sigma
        self.device = get_device()

        #init params
        rnn_num_hiddens = self.rnn.num_hiddens
        self.Whq = nn.Parameter(torch.rand((rnn_num_hiddens, self.vocab_size), device= self.device) * sigma)
        self.bq = nn.Parameter(torch.rand((1, self.vocab_size), device= self.device) * sigma)
    
    def one_hot(self, X):
        # original X shape is (batch_size, num_steps)
        # we want to encode its shape to (num_steps, batch_size, vocab_size)
        # TODO: WHY WE TRANSPOSE LIKE THIS?
        # https://d2l.ai/chapter_recurrent-neural-networks/rnn-scratch.html#one-hot-encoding
        # We often transpose the input so that we will obtain an output of 
        # shape (number of time steps, batch size, vocabulary size). 
        # This will allow us to loop more conveniently through the 
        # outermost dimension for updating hidden states of a minibatch, time step by time step
        return F.one_hot(X.T, self.vocab_size).type(torch.float)

    def forward(self, inputs):
        # print('inputs shape = ', inputs.shape)
        embedding = self.one_hot(inputs)
        # print('embedding shaep = ', embedding.shape)

        rnn_outputs, state = self.rnn(embedding)
        # print('rnn output shape = ', rnn_outputs.shape)
        # print('state output shape = ', state.shape)

        return self.output_layer(rnn_outputs)
    
    def output_layer(self, rnn_outputs):
        outputs = torch.stack([H @ self.Whq + self.bq for H in rnn_outputs], dim = 1)
        # print('LM output shape = ', outputs.shape)
        return outputs
    
    def train_step(self, input):
        pass

    def valid_step(self, input):
        pass

#Test correctness of LM

batch_size = 8
num_steps = 16
vocab_size = 1000
num_hiddens = 32

sample_inputs = torch.randint(0, vocab_size, (batch_size, num_steps), device = get_device())
rnn = RNN(vocab_size, num_hiddens)
lm  = LanguageModel(rnn, vocab_size)

outputs = lm(sample_inputs)
pred = torch.argmax(outputs[0], dim = 1)

print(pred)
print(pred.shape)

tensor([345, 345, 345, 345, 345, 345, 345, 345, 345, 345, 345, 345, 345, 345,
        345, 345], device='cuda:0')
torch.Size([16])


# Training procedure

In [13]:
if os.path.exists("dataset_object.pkl"):
    with open("dataset_object.pkl", "rb") as file:
        dataset = pickle.load(file)
else:
    dataset = ProjectGutenbergDataset(num_steps=1000)
    with open("dataset_object.pkl", "wb") as file:
        pickle.dump(dataset, file)

print(type(dataset))

PG dataset loaded
building vocab from 772176 tokens
built vocab object


In [17]:

vocab_size = len(dataset.vocab)
batch_size = 128 
num_hiddens = 32
num_steps = 1000


rnn = RNN(vocab_size, num_hiddens)
model = LanguageModel(rnn, vocab_size)

optim = torch.optim.SGD(model.parameters(), lr = 0.001)
loss = torch.nn.CrossEntropyLoss()

train_dataloader = DataLoader(dataset, batch_size = batch_size, shuffle= True)
print(len(train_dataloader))

epochs = 100

for e in range(epochs):
    print(f'epoch = {e}')
    for batch_id, (X, y) in enumerate(tqdm(train_dataloader)):
        X = X.to(device = get_device())
        y = y.to(device = get_device(), dtype= torch.float32)
        y_pred = model(X)
        y_pred = torch.argmax(y_pred, dim =2).to(torch.float32)

        # print(y.shape)
        # print(y_pred.shape)

        l = torch.tensor(loss(y_pred, y), requires_grad=True, device = get_device())
        optim.zero_grad()
        l.backward()
        if batch_id % 1000 == 0:
            print(l.item())
        optim.step()


PG dataset loaded
building vocab from 772176 tokens
built vocab object


KeyboardInterrupt: 

In [ ]:
class Trainer:
    def __init__(self, max_epochs = 100, lr = 0.001):
        self.max_epochs = max_epochs
        self.lr = lr
        self.device = 'cuda' if torch.cuda.device_count() else 'cpu'

    def fit(self, model, train_dataloader, valid_dataloader = None, optim = None, loss = None):
        if optim == None:
            optim = torch.optim.SGD(model.parameters(), lr = self.lr)
        
        if loss == None:
            def loss_function(y_hat, y):
                return F.cross_entropy(y_hat, y)

            loss = loss_function

        for batch_id, (X, y) in enumerate(train_dataloader):
            X = X.to(device = self.device)
            y = y.to(device = self.device, dtype= torch.float32)
            y_pred = model(X)
            y_pred = torch.argmax(y_pred, dim =2).to(torch.float32)
            print(y_pred.grad_fn)
            print(y.grad_fn)

            # print('batch id ', batch_id)
            # print(f'X: {X.shape}, device = {X.device}')
            # print(f'y: {y.shape}, device = {y.device}')
            # print('output shape = ', y_pred.shape)

            # train_loss = loss(y_pred, y)
            # print(type(train_loss))
            # train_loss.backward()
            # print(train_loss.item())

            break

trainer = Trainer()
print('dataset num steps = ', dataset.num_steps)

vocab_size = len(dataset.vocab)
print('vocab size ', vocab_size)
batch_size = 8 
num_hiddens = 16

train_dataloader = DataLoader(dataset, batch_size = batch_size, shuffle= True)

rnn = RNN(vocab_size, num_hiddens)
model = LanguageModel(rnn, vocab_size)

trainer.fit(model, train_dataloader=train_dataloader)




dataset num steps =  100
vocab size  13361
None
None


In [ ]:
y_hat = torch.rand((2,10,13361), dtype=torch.float32)
# print(y_hat)
# y_hat = torch.argmax(y_hat, dim = 2).to(torch.float32)
y = torch.rand((2,10), dtype=torch.float32)
print(y_hat.shape)
print(y_hat.dtype)
print(y.shape)

# l = F.cross_entropy(y_hat, y)
# l.backward()

with torch.no_grad():
    l = torch.nn.CrossEntropyLoss(y_hat, y)

    print(type(l))


torch.Size([2, 10, 13361])
torch.float32
torch.Size([2, 10])


RuntimeError: Boolean value of Tensor with more than one value is ambiguous